<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --upgrade implicit

In [7]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [6]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-6-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [4]:
len(df)

NameError: ignored

In [5]:
df.head(2)

NameError: ignored

# Data Preprocessing

In [ ]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)

In [ ]:
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [ ]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

In [ ]:
df[df["Qty Shipped"] == 0].head(50)

In [ ]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [ ]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [ ]:
len(df)

In [ ]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

## User and sku unqiue list

In [ ]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [ ]:
df_1 = df[["bcn_id","sku_id","Qty Shipped", "purchase"]]

In [ ]:
df_1.info()

In [ ]:
df_1.head(3)

In [ ]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "Qty Shipped": "sum",
    "purchase": "sum"
}).reset_index()

In [ ]:
grouped_df_binary = grouped_df[["bcn_id", "sku_id", "purchase"]]
grouped_df_binary["purchase"] = 1
grouped_df_binary

In [ ]:
grouped_df.sort_values(by="bcn_id",ascending=True)

In [ ]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()
skus

## Create sparse user x item and item x user matrices

In [ ]:
sparse_item_user = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['sku_id'], grouped_df['bcn_id'])))
sparse_user_item = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [ ]:
sparse_item_user_binary = sparse.csr_matrix((grouped_df_binary['purchase'].astype(float), (grouped_df_binary['sku_id'], grouped_df_binary['bcn_id'])))
sparse_user_item_binary = sparse.csr_matrix((grouped_df_binary['purchase'].astype(float), (grouped_df_binary['bcn_id'], grouped_df_binary['sku_id'])))

# **ALS Model**

In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

## LOOCV

with K=1 so 1 item per user gets dropped out

In [ ]:
x = leave_k_out_split(sparse_item_user, K=1)
sparse_item_user_train = x[0]
sparse_item_user_test = x[1]

In [ ]:
y = leave_k_out_split(sparse_user_item, K=1)
sparse_user_item_train = y[0]
sparse_user_item_test = y[1]

In [ ]:
y_binary = leave_k_out_split(sparse_user_item_binary, K=1)
sparse_user_item_train_binary = y_binary[0]
sparse_user_item_test_binary = y_binary[1]

## ALS Model param tuning

In [ ]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(sparse_user_item_train)

ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





In [ ]:
def validate(sparse_user_item_train, sparse_user_item_test, factors=200, alpha=40, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension)
    for <<iterations>> over matrices and validate with MAP@10
    """

    model = implicit.als.AlternatingLeastSquares(factors=factors,
                                                 alpha=alpha,
                                                 iterations=iterations,
                                                 regularization=regularization)

    model.fit(sparse_user_item_train, show_progress=show_progress)

    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@10 to a library that allows repeated items in prediction
    map10 = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Confidence: {alpha:>9} - Regularization: {regularization:4.3f} - Iterations: {iterations:>2} ==> MAP@10: {map10:6.5f}")
    return map10

In [ ]:
# %%time
# best_map10 = 0
# for factors in [10, 50, 100, 150, 200]:
#   for alpha in [10, 20, 30, 40]:
#     for iterations in [10, 15, 20]:
#       for regularization in [0.01, 0.05, 0.1]:
#           map10 = validate(sparse_user_item_train, sparse_user_item_test, factors, alpha, iterations, regularization, show_progress=False)
#           if map10 > best_map10:
#             best_map10 = map10
#             best_params = {'factors': factors, 'confidence': alpha, 'iterations': iterations, 'regularization': regularization}
#             print(f"Best MAP@10 found. Updating: {best_params}")

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV


class WrappedALS(BaseEstimator):
    def __init__(self, factors=100, regularization=0.01, iterations=20, alpha=40):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.alpha = alpha
        self.als_model = AlternatingLeastSquares(
            factors=self.factors,
            regularization=self.regularization,
            iterations=self.iterations,
            calculate_training_loss=True,
        )

    def fit(self, X, y=None):
        self.als_model.fit(X)
        return self

    def predict(self, X):
        return self.als_model.user_factors.dot(self.als_model.item_factors.T)

    def score(self, X, y=None):
        map_at_k = mean_average_precision_at_k(self.als_model, X, X.T, K=10)
        return map_at_k

# Define the hyperparameter ranges
param_grid = {
    'factors': [10, 50, 100, 150, 200],
    'alpha': [40],
    'iterations': [10, 20],
    'regularization': [0.01, 0.05]
}

# Initialize WrappedALS model
wrapped_als_model = WrappedALS()

# Initialize GridSearchCV
grid_search = GridSearchCV(wrapped_als_model, param_grid, cv=3, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(sparse_user_item_train)

# Get the best ALS model and its hyperparameters
best_als_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:")
print(best_params)

print("-----------------------------------------------------------------------------------------")
print("Best model MAP@10:")
best_model = AlternatingLeastSquares(**best_params)
map_10_best = mean_average_precision_at_k(best_model, sparse_user_item_train, sparse_user_item_test, K=10)
print(map_10_best)

In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=100,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10)

In [ ]:
model_2 = implicit.als.AlternatingLeastSquares(factors=10,
                                                 alpha=40,
                                                 iterations=10,
                                                 regularization=0.01)
model_2.fit(sparse_user_item_train)
map10_2 = mean_average_precision_at_k(model_2,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_3 = implicit.als.AlternatingLeastSquares(factors=50,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_3.fit(sparse_user_item_train)
map10_3 = mean_average_precision_at_k(model_3,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_4 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_4.fit(sparse_user_item_train)
map10_4 = mean_average_precision_at_k(model_4,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=15,
                                                 iterations=10,
                                                 regularization=0.05)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

## ALS Best Model Performance

In [ ]:
precision_als = precision_at_k(model_1, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

In [ ]:
ndcg = ndcg_at_k(model_1, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

## ALS Model Recommendations

In [ ]:
 # Assuming you want recommendations for user with ID 10
user_id = 5000
# Get the row of the user from the sparse_user_item matrix
user_row = sparse_user_item[user_id]
# Reshape the user_row to have 1 row per user
user_items = np.reshape(user_row, (1, -1))

# Now you can call the recommend function
recommended = model_1.recommend(user_id, user_items, N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab

In [ ]:
#Get similar items
item_id = 7
n_similar = 10
similar = model.similar_items(item_id, n_similar)
print(similar)

# BPR Model

In [ ]:
!pip install lightfm

In [ ]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score

In [ ]:
bpr_model = BPR(factors=200,
                learning_rate=0.01,
                iterations=20,
                verify_negative_samples=True)

In [ ]:
bpr_model.fit(sparse_user_item_train_binary, show_progress=True)

In [ ]:
mean_average_precision_at_k(bpr_model, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10)

In [ ]:
lightfm_bpr_model = LightFM(learning_rate=0.05, loss='bpr', no_components=100)
lightfm_bpr_model.fit(sparse_user_item_train_binary, epochs=10)

In [ ]:
train_precision = precision_at_k(lightfm_bpr_model, sparse_user_item_train_binary, k=10).mean()
test_precision = precision_at_k(lightfm_bpr_model, sparse_user_item_test_binary, k=10).mean()

In [ ]:
print(train_precision)

## BPR Model param tuning